### Header

In [1]:
# import libraries

# maths
import numpy as np
import pandas as pd
#import scipy.stats as stats
#from pandas.api.types import is_numeric_dtype

# visual
#from matplotlib_venn import venn2
#import seaborn as sns
#import matplotlib.pyplot as plt
#%matplotlib inline

# modelling
# from sklearn import linear_model
# from sklearn.linear_model import Ridge,LinearRegression,LogisticRegression
# from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
# from sklearn.preprocessing import StandardScaler,PolynomialFeatures
# from sklearn.metrics import r2_score,mean_squared_error,confusion_matrix,accuracy_score
# from sklearn.pipeline import make_pipeline,Pipeline

# nlp
# from sklearn.feature_extraction.text import CountVectorizer,HashingVectorizer,TfidfVectorizer
# from sklearn.naive_bayes import MultinomialNB
# from nltk.stem import PorterStemmer,WordNetLemmatizer
# from nltk.tokenize import sent_tokenize, word_tokenize
# from nltk.corpus import stopwords
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
# import spacy
# from spacy.tokens import Doc

# web
#import requests
#import json

# others
import os
import re
import time
import datetime as datetime

In [2]:
# file paths

input_path = '../data/2_input/'
clean_path = '../data/3_clean/'
output_path = '../data/4_output/'

image_path = '../images/'

### Functions

In [3]:
# split dates

def create_yr(x): 
    return x.split('-')[0] 

def create_mth(x): 
    return x.split('-')[1] 

def create_day(x): 
    return x.split('-')[2] 

def clean_date(df): 
    df['year'] = df.Date.apply(create_yr)
    df['month'] = df.Date.apply(create_mth)
    df['day'] = df.Date.apply(create_day)
    
    return df

In [4]:
def rename_columns (columns):
    return [column.lower() for column in columns]

In [5]:
def count_m(x):
    
    if x == 'M':
        return 1
    else:
        return 0

In [6]:
def count_dash(x):
    
    if x == '-':
        return 1
    else:
        return 0

In [7]:
def count_t(x):
    
    if x == '  T':
        return 1
    else:
        return 0

In [8]:
def replace_symbol(x):
    
    if x == 'M' or x == '-':
        return 'NaN'
    elif x=='  T':
        return 0
    else:
        return x

In [9]:
# count total number of M - T in df

def print_summary(df):    

    cols = ['column','M','-','T']
    df_summary = pd.DataFrame(columns=cols)
    idx = 0

    for col in df.columns:

        if df[col].dtype == 'object':

            total_m = df[col].apply(count_m).sum()
            total_dash = df[col].apply(count_dash).sum()
            total_t = df[col].apply(count_t).sum()

            df_summary.at[idx,cols[0]] = col
            df_summary.at[idx,cols[1]] = total_m
            df_summary.at[idx,cols[2]] = total_dash
            df_summary.at[idx,cols[3]] = total_t

        idx += 1
    
    return df_summary

### Import Data

In [10]:
# import weather data

df = pd.read_csv(input_path + 'weather.csv')

### Inspect Data

In [11]:
# print first 5 records

print(df.shape)
df.head()

(2944, 22)


,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [12]:
# list all columns

print(df.columns)

Index(['Station', 'Date', 'Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint',
       'WetBulb', 'Heat', 'Cool', 'Sunrise', 'Sunset', 'CodeSum', 'Depth',
       'Water1', 'SnowFall', 'PrecipTotal', 'StnPressure', 'SeaLevel',
       'ResultSpeed', 'ResultDir', 'AvgSpeed'],
      dtype='object')


In [13]:
# df summary

df.describe()

,Station,Tmax,Tmin,DewPoint,ResultSpeed,ResultDir
count,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000
mean,1.500000,76.166101,57.810462,53.457880,6.960666,17.494905
std,0.500085,11.461970,10.381939,10.675181,3.587527,10.063609
min,1.000000,41.000000,29.000000,22.000000,0.100000,1.000000
25%,1.000000,69.000000,50.000000,46.000000,4.300000,7.000000
50%,1.500000,78.000000,59.000000,54.000000,6.400000,19.000000
75%,2.000000,85.000000,66.000000,62.000000,9.200000,25.000000
max,2.000000,104.000000,83.000000,75.000000,24.100000,36.000000


In [14]:
# show df information

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 22 columns):
Station        2944 non-null int64
Date           2944 non-null object
Tmax           2944 non-null int64
Tmin           2944 non-null int64
Tavg           2944 non-null object
Depart         2944 non-null object
DewPoint       2944 non-null int64
WetBulb        2944 non-null object
Heat           2944 non-null object
Cool           2944 non-null object
Sunrise        2944 non-null object
Sunset         2944 non-null object
CodeSum        2944 non-null object
Depth          2944 non-null object
Water1         2944 non-null object
SnowFall       2944 non-null object
PrecipTotal    2944 non-null object
StnPressure    2944 non-null object
SeaLevel       2944 non-null object
ResultSpeed    2944 non-null float64
ResultDir      2944 non-null int64
AvgSpeed       2944 non-null object
dtypes: float64(1), int64(5), object(16)
memory usage: 506.1+ KB


In [15]:
# Check for nulls in columns

null_cols = df.isnull().sum()
mask_null = null_cols > 0
null_cols[mask_null].sort_values(ascending=False)

Series([], dtype: int64)

In [16]:
# Check for nulls in rows

null_rows = df.isnull().sum(axis=1)
mask_null = null_rows > 0
null_rows[mask_null].sort_values(ascending=False)

Series([], dtype: int64)

In [17]:
# check columns type

df.dtypes

Station          int64
Date            object
Tmax             int64
Tmin             int64
Tavg            object
Depart          object
DewPoint         int64
WetBulb         object
Heat            object
Cool            object
Sunrise         object
Sunset          object
CodeSum         object
Depth           object
Water1          object
SnowFall        object
PrecipTotal     object
StnPressure     object
SeaLevel        object
ResultSpeed    float64
ResultDir        int64
AvgSpeed        object
dtype: object

In [18]:
# count total number of M - T in df

print('before cleaning:')
df_summary = print_summary(df)
df_summary

before cleaning:


,column,M,-,T
1,Date,0,0,0
4,Tavg,11,0,0
5,Depart,1472,0,0
7,WetBulb,4,0,0
8,Heat,11,0,0
9,Cool,11,0,0
10,Sunrise,0,1472,0
11,Sunset,0,1472,0
12,CodeSum,0,0,0
13,Depth,1472,0,0


### Clean Data

In [19]:
for col in df.columns:
    
    df[col] = df[col].apply(replace_symbol)

print('after cleaning:')    
df_summary = print_summary(df)
df_summary

after cleaning:


,column,M,-,T
1,Date,0,0,0
4,Tavg,0,0,0
5,Depart,0,0,0
7,WetBulb,0,0,0
8,Heat,0,0,0
9,Cool,0,0,0
10,Sunrise,0,0,0
11,Sunset,0,0,0
12,CodeSum,0,0,0
13,Depth,0,0,0


In [20]:
# split dates

df = clean_date(df)
df.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,year,month,day
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,0.0,0.00,29.10,29.82,1.7,27,9.2,2007,05,01
1,2,2007-05-01,84,52,68,NaN,51,57,0,3,...,NaN,0.00,29.18,29.82,2.7,25,9.6,2007,05,01
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,0.0,0.00,29.38,30.09,13.0,4,13.4,2007,05,02
3,2,2007-05-02,60,43,52,NaN,42,47,13,0,...,NaN,0.00,29.44,30.08,13.3,2,13.4,2007,05,02
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,0.0,0.00,29.39,30.12,11.7,7,11.9,2007,05,03


In [21]:
# clean trailing space in codesum

for idx,cell in df['CodeSum'].iteritems():
    
    if cell == ' ':
        df.at[idx,'CodeSum'] = 'NaN'

In [22]:
# clean columns

df.drop(columns='Date',inplace=True)

df.columns = rename_columns(df.columns)                                      

new_order = ['year', 'month', 'day','station', 'tmax', 'tmin', 'tavg', 'depart', 'dewpoint', 'wetbulb',
               'heat', 'cool', 'sunrise', 'sunset', 'codesum', 'depth', 'water1',
               'snowfall', 'preciptotal', 'stnpressure', 'sealevel', 'resultspeed',
               'resultdir', 'avgspeed']

df = df[new_order]

### Output Data

In [23]:
df.head()

,year,month,day,station,tmax,tmin,tavg,depart,dewpoint,wetbulb,...,codesum,depth,water1,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,2007,05,01,1,83,50,67,14,51,56,...,NaN,0,NaN,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2007,05,01,2,84,52,68,NaN,51,57,...,NaN,NaN,NaN,NaN,0.00,29.18,29.82,2.7,25,9.6
2,2007,05,02,1,59,42,51,-3,42,47,...,BR,0,NaN,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2007,05,02,2,60,43,52,NaN,42,47,...,BR HZ,NaN,NaN,NaN,0.00,29.44,30.08,13.3,2,13.4
4,2007,05,03,1,66,46,56,2,40,48,...,NaN,0,NaN,0.0,0.00,29.39,30.12,11.7,7,11.9


In [24]:
# output clean data

df.to_csv(clean_path + 'weather_clean.csv',index=False)